# Address Geocoding Notebook
This notebook provides a detailed workflow for geocoding addresses from tabular data such as CSV files. It covers data import, address data cleaning, resolving unmatched addresses, visualizing results on a map, QC operations, and exporting results to GIS format. Nominatim and Google Maps API are the primary geocoders used however other ones may be used from the geopy.geocoders library. Please note you must have your own Google Maps API key to use that service. Nominatim does not require an API key. This notebook is based upon https://youtu.be/N6itC6hbOvo?si=gY-hNBpRieMCEYqu youtube video but modified with additional features and capabilities.

## 1. Import Packages and Libraries
These libraries may be installed into your Python virtual environment using pip: pandas, geopandas, leafmap, geopy

In [ ]:
import os
import re
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm

## 2. Create Data Folders
Unless a path is specified, these folders get created in the same location by default where your notebook Python file is stored

In [ ]:
data_folder = 'data' # where input data such as csv files are placed
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

## 3. Create Address Dataframe
Specify the name of a CSV file located in the data folder created from above. This file will then be imported into a new Pandas dataframe named address_df. Alternatively, a database connection may be created here to create the dataframe from a database table. This cell will output basic information about the new dataframe and the tabular view of the data.

In [ ]:
csv_file = 'Dialer_Export_1716925317652.csv'  # name of csv file
csv_file_path = os.path.join(data_folder, csv_file)
address_df = pd.read_csv(csv_file_path)
address_df.info()
address_df

## 4. Display Column Percent Complete
This cell will display the % complete and datatype for each column in the dataframe.

In [ ]:
completeness = (1 - address_df.isna().mean()) * 100
data_types = address_df.dtypes
column_info = pd.DataFrame({
    'Completeness (%)': completeness,
    'Data Type': data_types
})
column_info

## 5. Clean Address Data
Clean address columns if necessary. The below sample code provides techniques which may be used to clean address data if required. Additional code may be added as needed. This code and its use is explained in more detail in the youtube video.

In [ ]:
#def make_ordinal(match):
    #n = int(match.group(1))
    #if 11 <= (n % 100) <= 13:
        #suffix = 'th'
    #else:
        #suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    #return str(n) + suffix + match.group(2)
    
#def update_address(row):
    #old_address = row['Address']
    #pattern = r'(\d+)(\s+(?:Street|Avenue|Blvd|Drive))'
    #result = re.sub(pattern, make_ordinal, old_address)

#address_df['address_fixed'] = address_df.apply(update_address, axis=1)

#address_df = address_df.head(250) # just to shorten the dataset for testing / dev

# Check for duplicate addresses if needed

## 6. Create Full Address Column
Address columns are concatenated together to create a full_address column. If addresses are already in a single column, this operation will not be needed. Trailing whitespace is removed and the column is displayed. Use openstreetmap.org (which powers Nominatim) to manually geocode a few addresses from the full_address column for test. If geocoding fails, additional cleaning may be required.

In [ ]:
address_df['full_address'] = (
    address_df['Address'] + ',' + 
    address_df['City'] + ',' +
    address_df['State'] + ',' + 
    address_df['Zip'].astype(str)
)
# Removing any leading and trailing whitespace
address_df['full_address'] = address_df['full_address'].str.strip()
address_df['full_address']

## 7. Geocode Using Nominatim
Geocode address_df dataframe using Nominatim. This process creates a new column in the dataframe named location which stores a location object if the address successfully geocodes. If the address fails to geocode, a None value will be stored in the location column. The geocoder may be tested on single addresses prior to running the whole dataframe if needed (see the commented code and make sure to uncomment the last line which geocodes the whole dataframe). A progress bar will appear below the cell to indicate geocoding progress. The Nominatim API only allows 1 request per and is therefore very slow for large datasets. For faster performance, use a local installation of Nominatim.

In [ ]:
tqdm.pandas()
locator = Nominatim(user_agent='justinhawley', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1) # set rate limiter as Nominatim can only receive 1 request per second

# Test the geocoder
#geocode('1422 N 35TH ST ,MILWAUKEE,Wisconsin,53208')

address_df['location'] = address_df['full_address'].progress_apply(geocode)

## 8. Check Unmatched Records (Nominatim)
Check for any null location objects for addresses that failed to geocode. Percentage unmatched is reported and a new dataframe is created for any unmatched records. The unmatched records are then removed from the address_df dataframe so it only contains matched records. The total matched and unmatched record counts should equal the original dataset total.

In [ ]:
nominatim_unmatched_df = address_df[address_df['location'].isna()] # create new dataframe for unmatched
nominatim_unmatched_percent = (len(nominatim_unmatched_df) / len(address_df)) * 100.0
print(f'{len(nominatim_unmatched_df)} ({nominatim_unmatched_percent}%) unmatched records')

In [ ]:
# Create new matched only datafram (nominatim_matched_df) from address_df
if len(nominatim_unmatched_df) > 0:
    nominatim_matched_df = address_df[-address_df['location'].isna()] # create matched dataframe
else:
    nominatim_matched_df = address_df

# Verify unmatched records removed from address_df
print(len(nominatim_unmatched_df), 'unmatched records.', len(nominatim_matched_df), 'matched records', 'for a total of:',len(nominatim_matched_df) + len(nominatim_unmatched_df))

## 9. Geocode unmatched records with Google Maps API
The remaining unmatched records may be geocoded with Google Maps API or any other geocoding service. You must have your own API key to use Google Maps API. Matched records are placed into a new dataframe named address_df2.

In [ ]:
api_key = 'YOUR GOOGLE MAPS API KEY' 
locator = GoogleV3(api_key=api_key, timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1) # may change to 5

# test the geocoder
# geocode('1422 N 35TH ST ,MILWAUKEE,Wisconsin,53208')
tqdm.pandas()  # Adds progress bar for apply

#attempt to geocode unmatched dataframe
nominatim_unmatched_df['location'] = nominatim_unmatched_df['full_address'].progress_apply(geocode) #original line
google_matched_df = nominatim_unmatched_df # create new dataframe for readibility

## 10. Check Unmatched Again
Check for unmatched records again after the Google Maps API geocode. Remaining unmatched records are placed in google_unmatched_df dataframe

In [ ]:
# Query google_matched_df (from above geocode) to see if there are still more unmatched
google_unmatched_df = google_matched_df[google_matched_df['location'].isna()]
google_unmatched_percent = (len(google_unmatched_df) / len(google_matched_df)) * 100.0
print(f'{len(google_unmatched_df)} ({google_unmatched_percent}%) unmatched records')

In [ ]:
# If Unmatched records, remove from google_matched_df
if len(google_unmatched_df) > 0:
    google_matched_df = google_matched_df[-google_matched_df['location'].isna()]

# Verify unmatched records removed from google_matched_df. Dataframe will be empty if there was no matches from Google API geocode
len(google_matched_df)

## 11. Add Lat Long Columns Verify Matched and Unmatched Dataframe Counts
Verify the dataframe record counts for Nominatim and Google API geocoding results

In [ ]:
# Create lat long columns
google_matched_df['latitude'] = google_matched_df['location'].apply(lambda loc: loc.latitude if loc else None)
google_matched_df['longitude'] = google_matched_df['location'].apply(lambda loc: loc.longitude if loc else None)

google_unmatched_df['latitude'] = google_unmatched_df['location'].apply(lambda loc: loc.latitude if loc else None)
google_unmatched_df['longitude'] = google_unmatched_df['location'].apply(lambda loc: loc.longitude if loc else None)

nominatim_matched_df['latitude'] = nominatim_matched_df['location'].apply(lambda loc: loc.latitude if loc else None)
nominatim_matched_df['longitude'] = nominatim_matched_df['location'].apply(lambda loc: loc.longitude if loc else None)

nominatim_unmatched_df['latitude'] = nominatim_unmatched_df['location'].apply(lambda loc: loc.latitude if loc else None)
nominatim_unmatched_df['longitude'] = nominatim_unmatched_df['location'].apply(lambda loc: loc.longitude if loc else None)
# Google unmatched is the final unmatched count
print(f'Nominatim matched: {len(nominatim_matched_df)} \nNominatim unmatched: {len(nominatim_unmatched_df)}\nGoogle matched: {len(google_matched_df)}\nGoogle unmatched: {len(google_unmatched_df)}')

## 12. Manually Correct Remaining Unmatched Records
If possible, manually add latitude and longitude values for any remaining unmatched records in unmatched2_df dataframe.

In [ ]:
google_unmatched_df

In [ ]:
# Manually assign lat long values to each unmatched record
# Make sure to use correct index from above
google_unmatched_df.loc[5994,['latitude', 'longitude']] = (46.015715937394, -91.48644435376464) #updates row 5994 from above unmatched

# Add updated record to google_matched_df and remove it from google_unmatched_df 
updated_record = google_unmatched_df.loc[5994]
updated_record = updated_record.to_frame().T
google_unmatched_df = google_unmatched_df.drop(5994)
google_matched_df = pd.concat([google_matched_df, updated_record], ignore_index=True, axis=0)

# Verify final dataframe counts
print(f'Nominatim matched: {len(nominatim_matched_df)} \nNominatim unmatched: {len(nominatim_unmatched_df)}\nGoogle matched: {len(google_matched_df)}\nGoogle unmatched: {len(google_unmatched_df)}')

## 12. Combined Matched Dataframes
Combine matched matched records into a new single dataframe. 

In [ ]:
matched_final_df = pd.concat([nominatim_matched_df, google_matched_df], axis=0)
matched_final_df = matched_final_df.reset_index(drop=True)

unmatched_final_df = google_unmatched_df # google_unmatched_df is the final unmatched. It is renamed to unmatched_final for readability

#len(matched_final)
matched_final_df

In [ ]:
# Verify address_df_merged has no null lat long values
matched_final_df[matched_final_df['latitude'].isna() | matched_final_df['longitude'].isna()]

## 14. Create Geometry and Geodataframe
Create a geometry column and a geodataframe for the final unmatched


In [ ]:
geometry = gpd.points_from_xy(matched_final_df.longitude, matched_final_df.latitude)
matched_final_gdf = gpd.GeoDataFrame(matched_final_df, crs='EPSG:4326', geometry = geometry)
matched_final_gdf

## 15. Create Geometry and Geodataframe

In [ ]:
# Remove the location column as it throws an error when displaying the below map (TypeError: Object of type Location is not JSON serializable)
del matched_final_gdf['location']

# Correct any int64 columns which throws the same error
def make_json_serializable(val):
    if isinstance(val, (pd.Int64Dtype, pd.Float64Dtype)):
        return val.item()
    return val

matched_final_gdf = matched_final_gdf.applymap(make_json_serializable)

In [ ]:
# Visualize results
m = leafmap.Map(width=800, height=500)
matched_final_gdf.explore(m=m)
m.zoom_to_gdf(matched_final_gdf)
m

## 16. QC Final Matched
More QC operations may be added here. Currently this code checks for groups of coincident points and reports of they have the same or different addresses. Additionally it might be valuable to have the option of removing duplicate addresses prior to geocoding.

In [ ]:
coincident_points = matched_final_gdf[matched_final_gdf.duplicated(subset='geometry', keep=False)]

# Step 2: Group by geometries
groups = coincident_points.groupby(coincident_points.geometry.apply(lambda geom: geom.wkt))

# Step 3: Check for same or different addresses in each group
found_different_addresses = False
for geom_wkt, group in groups:
    unique_addresses = group['full_address'].unique()
    if len(unique_addresses) == 1:
        print(f"All {len(group)} points at {geom_wkt} have the same address: {unique_addresses[0]}")
        print()
    else:
        print(f"Points at {geom_wkt} have different addresses: {unique_addresses}. Count: {len(group)}")
        print()
        found_different_addresses = True

if not found_different_addresses:
    print("No groups with different addresses found.")


## 17. Export Matched and Unmatched

In [ ]:
# Export matched to shapefile
output_file = csv_file.replace('.csv', '_geocode.shp')
output_path = os.path.join(output_folder, output_file)
matched_final_gdf .to_file(filename=output_path)

In [ ]:
# Export unmatched to csv file
output_file = csv_file.replace('.csv', '_unmatched.csv')
output_path = os.path.join(output_folder, output_file)
unmatched_final_df.to_csv(output_path)